In [1]:
#!pip install glfw
#!pip install pyopengl
import glfw
from OpenGL.GL import *
import OpenGL.GL.shaders
import numpy as np


In [2]:

def f(altura, raio, qtd, rotacao_inicial=0):
    angulos = np.linspace(0+rotacao_inicial, 2 * np.pi + rotacao_inicial, qtd, endpoint=False)
    pontos = [(raio * np.cos(ang), altura,raio * np.sin(ang)) for ang in angulos]
    return pontos

quad_inf = f(0,0.5,4, np.pi/4)
quad_sup = f(1,0.5,4, np.pi/4)

#cubo
objeto_1 = [
    # Face inferior (base do cubo)
    quad_inf[0], quad_inf[3], quad_inf[2],  # invertido
    quad_inf[0], quad_inf[2], quad_inf[1],

    # Face superior (topo do cubo)
    quad_sup[0], quad_sup[2], quad_sup[3],
    quad_sup[0], quad_sup[1], quad_sup[2],  # invertido

    # Face lateral 1 (entre quad_inf[0] e quad_sup[0])
    quad_inf[0], quad_inf[1], quad_sup[0],  # invertido
    quad_inf[1], quad_sup[1], quad_sup[0],

    # Face lateral 2 (entre quad_inf[1] e quad_sup[1])
    quad_inf[1], quad_inf[2], quad_sup[1],  # invertido
    quad_inf[2], quad_sup[2], quad_sup[1],

    # Face lateral 3 (entre quad_inf[2] e quad_sup[2])
    quad_inf[2], quad_inf[3], quad_sup[2],  # invertido
    quad_inf[3], quad_sup[3], quad_sup[2],

    # Face lateral 4 (entre quad_inf[3] e quad_sup[3])
    quad_inf[3], quad_inf[0], quad_sup[3],  # invertido
    quad_inf[0], quad_sup[0], quad_sup[3]
]



#diamanate
quad= f(1,1/5, 4, np.pi/4)
octa= f(2/3,2/5, 8,np.pi/4)
zero = (0,0,0)

objeto_2 = [ 
    #abc
    quad[0],quad[2],quad[1],
    quad[2],quad[0],quad[3],
    
    quad[0], octa[1], octa[0],
    quad[0], quad[1], octa[1],
    quad[1],octa[2],octa[1],
    quad[1], octa[3], octa[2],
    quad[1], quad[2], octa[3],
    quad[2], octa[4], octa[3],
    quad[2],octa[5],octa[4],
    quad[2],quad[3],octa[5],
    quad[3],octa[6],octa[5],
    quad[3], octa[7], octa[6],
    quad[3],quad[0],octa[7],
    quad[0],octa[0],octa[7],

    octa[0],octa[1],zero,
    octa[1],octa[2],zero,
    octa[2],octa[3],zero,
    octa[3],octa[4],zero,
    octa[4],octa[5],zero,
    octa[5],octa[6],zero,
    octa[6],octa[7],zero,
    octa[7],octa[0],zero
    
]
objetos = []
objetos.append(objeto_1)
objetos.append(objeto_2)

#------------------------------------------------NÃO EDITAR DAQUI PRA BAIXO.
#cria uma lista com todos os vértices
vertices__ = []
for obj in objetos:
    for coord in obj:
        vertices__.append(coord)
qtd_vertices = len(vertices__)


vertices = np.zeros(len(vertices__), [("position", np.float32, 3)])
vertices['position'] = vertices__

#matrizes base
def m_i():
    return np.array([   1.0,  0.0, 0.0, 0.0, 
                        0.0,  1.0, 0.0, 0.0, 
                        0.0,  0.0, 1.0, 0.0, 
                        0.0,  0.0, 0.0, 1.0], np.float32)

def m_tr(tx,ty,tz):
    return np.array([      1.0,  0.0, 0.0, tx, 
                           0.0,  1.0, 0.0, ty, 
                           0.0,  0.0, 1.0, tz, 
                           0.0,  0.0, 0.0, 1.0], np.float32)
def m_rot_z(t):
    return np.array([     math.cos(t), -math.sin(t),   0.0, 0.0, 
                          math.sin(t),  math .cos(t),  0.0, 0.0, 
                          0.0,          0.0,           1.0, 0.0, 
                          0.0,          0.0,           0.0, 1.0], np.float32)

def m_rot_x(t):
    return np.array([     1.0,   0.0,        0.0,         0.0, 
                          0.0, math.cos(t), -math.sin(t), 0.0, 
                          0.0, math.sin(t),  math.cos(t), 0.0, 
                          0.0,   0.0,        0.0,         1.0], np.float32)

def m_rot_y(t):
    return np.array([     math.cos(t),  0.0, math.sin(t), 0.0, 
                          0.0,          1.0,   0.0,       0.0, 
                          -math.sin(t), 0.0, math.cos(t), 0.0, 
                          0.0,          0.0,   0.0,       1.0], np.float32)

def m_s(sx,sy,sz):
    return np.array([          sx,  0.0, 0.0, 0.0, 
                              0.0,  sy, 0.0, 0.0, 
                                    0.0,  0.0, sz, 0.0, 
                                    0.0,  0.0, 0.0, 1.0], np.float32)



glfw.init()
glfw.window_hint(glfw.VISIBLE, glfw.FALSE);
window = glfw.create_window(900, 700, "CG2024 - Projeto 1", None, None)
glfw.make_context_current(window)


vertex_code = """
        attribute vec3 position;
        uniform mat4 mat_transformation;
        void main(){
            gl_Position = mat_transformation * vec4(position,1.0);
        }
        """

fragment_code = """
        uniform vec4 color;
        void main(){
            gl_FragColor = color;
        }
        """

# Request a program and shader slots from GPU
program  = glCreateProgram()
vertex   = glCreateShader(GL_VERTEX_SHADER)
fragment = glCreateShader(GL_FRAGMENT_SHADER)


# Set shaders source
glShaderSource(vertex, vertex_code)
glShaderSource(fragment, fragment_code)

# Compile shaders
glCompileShader(vertex)
if not glGetShaderiv(vertex, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(vertex).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Vertex Shader")

glCompileShader(fragment)
if not glGetShaderiv(fragment, GL_COMPILE_STATUS):
    error = glGetShaderInfoLog(fragment).decode()
    print(error)
    raise RuntimeError("Erro de compilacao do Fragment Shader")

# Attach shader objects to the program
glAttachShader(program, vertex)
glAttachShader(program, fragment)


# Build program
glLinkProgram(program)
if not glGetProgramiv(program, GL_LINK_STATUS):
    print(glGetProgramInfoLog(program))
    raise RuntimeError('Linking error')
    
# Make program the default program
glUseProgram(program)


# Request a buffer slot from GPU
buffer = glGenBuffers(1)
# Make this buffer the default one
glBindBuffer(GL_ARRAY_BUFFER, buffer)

# Upload data
glBufferData(GL_ARRAY_BUFFER, vertices.nbytes, vertices, GL_DYNAMIC_DRAW)
glBindBuffer(GL_ARRAY_BUFFER, buffer)


# Bind the position attribute
# --------------------------------------
stride = vertices.strides[0]
offset = ctypes.c_void_p(0)

loc = glGetAttribLocation(program, "position")
glEnableVertexAttribArray(loc)


glVertexAttribPointer(loc, 3, GL_FLOAT, False, stride, offset)

loc_color = glGetUniformLocation(program, "color")


In [ ]:
#controles
tx = 0
ty = 0
tz = 0
ang_rot_x = 0
ang_rot_y = 0
ang_rot_z = 0
sx = 1
sy = 1
sz = 1


def key_event(window,key,scancode,action,mods):
    global tx, ty, ang_rot_x, ang_rot_y, ang_rot_z, sx,sy,sz
    
    print('[key event] key=',key)
    print('[key event] scancode=',scancode)
    print('[key event] action=',action)
    print('[key event] mods=',mods)
    print('-------')
    if key == 265: ty += 0.1 #cima #seta cima
    if key == 264: ty -= 0.1 #baixo #seta_baixo
    if key == 263: tx -= 0.1 #esquerda #seta esq
    if key == 262: tx += 0.1 #direita
        
    # teclas a, s
    if key == 65: ang_rot_x += 0.1 #rotacao x
    if key == 83: ang_rot_x -= 0.1 #rotacao x

    #teclas q,w
    if key == 81: ang_rot_y += 0.1
    if key == 87: ang_rot_y -= 0.1
        
    # teclas z , x
    if key == 90: 
        sx += 0.05 #aumenta escala
        sy += 0.05 #aumenta escala
        sz += 0.05 #aumenta escala
    if key == 88: 
        sx -= 0.05 #aumenta escala
        sy -= 0.05 #aumenta escala
        sz -= 0.05 #aumenta escala
    
glfw.set_key_callback(window,key_event)

glfw.show_window(window)

#------------------------------------------------------------------------------- NÂO EDITAR DAQUI PRA BAIXO.
import math
import random
d = 0.0
glEnable(GL_DEPTH_TEST) ### importante para 3D

def multiplica_matriz(a,b):
    m_a = a.reshape(4,4)
    m_b = b.reshape(4,4)
    m_c = np.dot(m_a,m_b)
    c = m_c.reshape(1,16)
    return c

def renderizar_objeto(primeiro_vertice,qtd_vertices, matriz_transformacao, strip_triangle="triangle",R=1/2, G=1/2, B=1/2):
    glUniformMatrix4fv(loc, 1, GL_TRUE, matriz_transformacao)

    if strip_triangle=="triangle":
        
        for triangle in range(0, qtd_vertices, 3):
            random.seed(triangle)            
            
            glUniform4f(loc_color, R, G, B, 1.0)
            glDrawArrays(GL_TRIANGLES, triangle+primeiro_vertice, 3)    
    elif strip_triangle == "strip":
        for retangulo in range(0,qtd_vertices, 4):
            random.seed(retangulo)
            R = random.random()
            G = random.random()
            B = random.random()
            glUniform4f(loc_color, R, G, B, 1.0)
            glDrawArrays(GL_TRIANGLE_STRIP, retangulo+primeiro_vertice, 4)    
    else:
        pass

def matriz_transform(tx, ty, tz, r_x, r_y, r_z, s_x, s_y, s_z):
    mat_translate = m_tr(tx, ty, tz)
    mat_rot_z = m_rot_z(r_z)
    mat_rot_x = m_rot_x(r_x)
    mat_rot_y = m_rot_y(r_y)
    mat_scale = m_s(s_x, s_y, s_z)
    
    mat_transform = multiplica_matriz(mat_rot_x, mat_rot_y) 
    mat_transform = multiplica_matriz(mat_transform, mat_rot_z) 
    mat_transform = multiplica_matriz(mat_transform,mat_scale)
    # print(mat_transform)
    mat_transform = multiplica_matriz(mat_transform, mat_translate)
        
    return mat_transform

def mult_n_matrizes(array_matrizes):
    '''
    É importante conferir que as matrizes são multiplicadas da esquerda pra direita e são aplicadas pela direita
    '''
    # Verifica se o array de matrizes não está vazio
    if len(array_matrizes) == 0:
        raise ValueError("O array de matrizes está vazio.")
    
    # Inicializa o resultado com a primeira matriz e garante que seja 4x4
    resultado = np.atleast_2d(array_matrizes[0]).reshape(4, 4)
    
    # Multiplica todas as matrizes subsequentes
    for matriz in array_matrizes[1:]:
        matriz = np.atleast_2d(matriz).reshape(4, 4)  # Garante que cada matriz seja 4x4
        
        resultado = np.matmul(resultado, matriz)
    
    return resultado



counter = 0
while not glfw.window_should_close(window):
    
    cont = 0
    glfw.poll_events() 
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT)
    glClearColor(1.0, 1.0, 1.0, 1.0)
    #----------------------------------------------------------------------------
    
    #----------------------------------------------------------------------------
    loc = glGetUniformLocation(program, "mat_transformation")
    # -----------------------------
    # Matriz de transformação geral
    
    
    dt = counter*np.pi/1800
    distancia_entre_objs = 1/3
    m1 = matriz_transform(0, 0 , 0,
                          0, 0, 0, 
                          1/5, 1/5, 1/5)
    # m2 = matriz_transform(distancia_entre_objs,0,0,
    #                       0, (2/3)*dt, 0, 
    #                       1/5, 1/5, 1/5)

    # m3 = matriz_transform(-distancia_entre_objs,0,0,
    #                       0, (3/2)*dt, 0, 
    #                       1/5, 1/5, 1/5)

    #diamantes
    m_d1 = mult_n_matrizes([m_tr(-1/3,0,0), m_tr(0,1/16,0), m_rot_y(dt/2), m_s(1/5,1/5,1/5)])
    m_d2 = mult_n_matrizes([m_tr(0,0,0)   , m_tr(0,1/16,0), m_rot_y(dt/2), m_s(1/5,1/5,1/5)])
    m_d3 = mult_n_matrizes([m_tr(1/3,0,0) , m_tr(0,1/16,0), m_rot_y(dt/2), m_s(1/5,1/5,1/5)])

    #mesa
    m_m1 = mult_n_matrizes([m_s(1.5, 1/16, 1.5)])
    m_m2 = mult_n_matrizes([m_tr(-1/2, -1/2,  -1/2),  m_s(1/16, 1/2, 1/16)])
    m_m3 = mult_n_matrizes([m_tr(-1/2, -1/2,   1/2),  m_s(1/16, 1/2, 1/16)])
    m_m4 = mult_n_matrizes([m_tr(1/2,  -1/2,  -0.5),  m_s(1/16, 1/2, 1/16)])
    m_m5 = mult_n_matrizes([m_tr(1/2,  -1/2,   0.5),  m_s(1/16, 1/2, 1/16)])

    #mão
    altura_mao = -5/32+np.sin(10*dt)/100
    m_h1 = mult_n_matrizes([m_tr(0, 1/2+altura_mao ,-29/64) ,  m_s(1/16, 1/16, 4/12)])
    m_h2 = mult_n_matrizes([m_tr(0, 1/2+altura_mao ,-1/4) ,    m_s(1/4, 1/16, 1/4)])
    
    m_h3 = mult_n_matrizes([m_tr(-3/32, 33/64+altura_mao ,-2/16) ,m_s(1/32, 1/32, 1/8)])
    m_h4 = mult_n_matrizes([m_tr(-1/32, 33/64+altura_mao ,-2/16) ,m_s(1/32, 1/32, 1/8)])
    m_h5 = mult_n_matrizes([m_tr(1/32,  33/64+altura_mao ,-2/16)  ,m_s(1/32, 1/32, 1/8)])
    m_h6 = mult_n_matrizes([m_tr(3/32,  33/64+altura_mao ,-2/16)  ,m_s(1/32, 1/32, 1/8)])

    m_h7 = mult_n_matrizes([m_tr(-3/32, 33/64+altura_mao ,-1/16) ,                  m_s(1/32, 1/32, 1/8)])
    m_h8 = mult_n_matrizes([m_tr(-1/32, 31/64+altura_mao ,-1/16), m_rot_x(np.pi/4), m_s(1/32, 1/32, 1/8)])
    m_h9 = mult_n_matrizes([m_tr(1/32,  31/64+altura_mao , -1/16)  ,m_rot_x(np.pi/4), m_s(1/32, 1/32, 1/8)])
    m_h10 = mult_n_matrizes([m_tr(3/32, 31/64+altura_mao ,-1/16)  ,m_rot_x(np.pi/4), m_s(1/32, 1/32, 1/8) ])

    
    m_h11 = mult_n_matrizes([m_tr(-7/64, 33/64+altura_mao ,-4/16) ,m_rot_y(-np.pi/6),m_s(1/32, 1/32, 1/8)])
    m_h12 = mult_n_matrizes([m_tr(-9/64, 33/64+altura_mao ,-5/32) , m_s(1/32, 1/32, 1/8)])
    
    #inclinação é aplicada à todos
        
    m_inclina = mult_n_matrizes([m_rot_x(-1/8), m_rot_y(dt/8)])
    # m_inclina = mult_n_matrizes([m_rot_y(-np.pi*1/32) ,m_rot_x(-np.pi*1/2)])
    # m_inclina = mult_n_matrizes([m_rot_y(-np.pi*1/32) ,m_rot_x(-np.pi*1/16)])
    # m_inclina = mult_n_matrizes([m_rot_y(np.pi*1/16), m_rot_x(-np.pi*1/32)])
    # m_inclina = mult_n_matrizes([m_i()])
 
    R = (random.random()*2-1)/10
    
    renderizar_objeto(len(objeto_1),len(objeto_2), mult_n_matrizes([m_inclina, m_d1]), R=0.7+R, G=0.7+R, B=1)
    renderizar_objeto(len(objeto_1),len(objeto_2), mult_n_matrizes([m_inclina, m_d2]),R=0.7+R, G=0.7+R, B=1)
    renderizar_objeto(len(objeto_1),len(objeto_2), mult_n_matrizes([m_inclina, m_d3]),R=0.7+R, G=0.7+R, B=1)

    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_m1]))
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_m2]))
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_m3]))
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_m4]))
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_m5]))

    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h1]), R=1)
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h2]), R=1)
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h3]), R=1)
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h4]), R=1)
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h5]), R=1)
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h6]), R=1)
    
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h7]), R=1)
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h8]), R=1)
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h9]), R=1)
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h10]), R=1)

    
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h11]), R=1)
    renderizar_objeto(0,len(objeto_1), mult_n_matrizes([m_inclina ,m_h12]), R=1)
    

    # renderizar_objeto(len(objeto_1),len(objeto_2), m2)
    
    # renderizar_objeto(len(objeto_1),len(objeto_2), m3)

    #mesa
    #==
    # renderizar_objeto(0,len(objeto_1),m1)
    # renderizar_objeto(0,len(objeto_1),m5,"strip")
    # renderizar_objeto(0,len(objeto_1),m6,"strip")
    # renderizar_objeto(0,len(objeto_1),mat_transform4,"strip")
    # renderizar_objeto(0,len(objeto_1),mat_transform1,"strip")
    # renderizar_objeto(len(objeto_1)+len(objeto_2),len(objeto_3),mat_transform3)
    
    #----------------------------------------------------------------------------


    
    glfw.swap_buffers(window)
    counter += 1

glfw.terminate()

